In [1]:
import pandas as pd

transform from all divisions and year into 1 dataframe for each country

In [ ]:
import os
import pandas as pd

root_dir = "data"
folder_dfs = {}  # Nested dict: country -> subfolder -> CSV -> DataFrame

# Walk through all directories
for dirpath, dirnames, filenames in os.walk(root_dir):
    if dirpath == root_dir:
        continue  # skip root

    # Split the path components relative to data/
    rel_path = os.path.relpath(dirpath, root_dir).split(os.sep)

    # Expecting: ["england", "0"] or ["england", "1"], etc.
    if len(rel_path) != 2:
        continue  # skip if not exactly two levels below root

    country, subfolder = rel_path

    # Initialize nested dicts
    folder_dfs.setdefault(country, {})
    folder_dfs[country].setdefault(subfolder, {})

    # Loop through CSV files
    for filename in filenames:
        if filename.endswith(".csv"):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)

            csv_name = os.path.splitext(filename)[0]  # e.g. "1920"
            folder_dfs[country][subfolder][csv_name] = df

# Check the structure
for country, subfolders in folder_dfs.items():
    print(f"\nCountry: {country}")
    for sub, csvs in subfolders.items():
        print(f"Subfolder: {sub}")
        for name, df in csvs.items():
            print(f"    - {name}.csv → shape={df.shape}")


def concat_country_data(country_name):
    """Concatenate all CSVs for a given country (across all subfolders)."""
    # Flatten all subfolder DataFrames into one list
    dfs = []
    for subfolder in folder_dfs[country_name].values():
        dfs.extend(subfolder.values())
    # Combine them into one big DataFrame
    return pd.concat(dfs, ignore_index=True)

# Build DataFrames per country
belgium_data     = concat_country_data("belgium")
england_data     = concat_country_data("england")
france_data      = concat_country_data("france")
germany_data     = concat_country_data("germany")
greece_data      = concat_country_data("greece")
italy_data       = concat_country_data("italy")
netherlands_data = concat_country_data("netherlands")
portugal_data    = concat_country_data("portugal")
scotland_data    = concat_country_data("scotland")
spain_data       = concat_country_data("spain")
turkey_data      = concat_country_data("turkey")



FILEPATH=""
country_dfs = {
    "belgium": belgium_data,
    "england": england_data,
    "france": france_data,
    "germany": germany_data,
    "greece": greece_data,
    "italy": italy_data,
    "netherlands": netherlands_data,
    "portugal": portugal_data,
    "scotland": scotland_data,
    "spain": spain_data,
    "turkey": turkey_data
}


output_dir = "combined_csvs"  # folder to save all CSVs
os.makedirs(output_dir, exist_ok=True)  # create folder if it doesn't exist

for country, df in country_dfs.items():
    # Safe file path
    file_path = os.path.join(output_dir, f"{country}.csv")
    
    df.to_csv(file_path, index=False)
    print(f" Saved {file_path} (shape={df.shape})")

all the single countries import for all years and all divisions

In [ ]:
#this import a bit useless since data is already here, but in case of separation its good 
#import of data
belgium_data=pd.read_csv(r'combined_csvs\belgium.csv')
eng_data = pd.read_csv(r'combined_csvs\england.csv')
fr_data=pd.read_csv(r'combined_csvs\france.csv')
d_data=pd.read_csv(r'combined_csvs\germany.csv')
gr_data=pd.read_csv(r'combined_csvs\greece.csv')
it_data=pd.read_csv(r'combined_csvs\italy.csv')
ne_data=pd.read_csv(r'combined_csvs\netherlands.csv')
por_data=pd.read_csv(r'combined_csvs\portugal.csv')
sc_data=pd.read_csv(r'combined_csvs\scotland.csv')
sp_data=pd.read_csv(r'combined_csvs\spain.csv')
tur_data=pd.read_csv(r'combined_csvs\turkey.csv')

C:\Users\vojte\AppData\Local\Temp\ipykernel_23956\3485001040.py:10: DtypeWarning: Columns (70) have mixed types. Specify dtype option on import or set low_memory=False.
  sc_data=pd.read_csv(r'combined_csvs\scotland.csv')


# function creates the small dataset(s) for A0 model

In [ ]:
col_list_original=['Div','Date','Time','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
def original_data(df,feature_list):
    col_list=feature_list
    df=df[col_list]
    df[['Country', 'Division']] =df['Div'].str.extract(r'([A-Za-z]+)(\d+)')
    df.drop(columns=['Div'], inplace=True)
    df['Total_goals']=df['FTHG']+df['FTAG']
    df['Target']= (df['Total_goals']>2.5).astype(int) #1 if more than 2 goals were scored, else 0
    return df


belgium_data= original_data(belgium_data,col_list_original)
eng_data = original_data(eng_data,col_list_original)
fr_data=original_data(fr_data,col_list_original)
d_data=original_data(d_data,col_list_original)
gr_data=original_data(gr_data,col_list_original)
it_data=original_data(it_data,col_list_original)
ne_data=original_data(ne_data,col_list_original)
por_data=original_data(por_data,col_list_original)
sc_data=original_data(sc_data,col_list_original)
sp_data=original_data(sp_data,col_list_original)
tur_data=original_data(tur_data,col_list_original)

data_og=pd.DataFrame()

for df in [belgium_data, eng_data, fr_data, d_data, gr_data, it_data, ne_data, por_data, sc_data, sp_data, tur_data]:
    data_og=pd.concat([data_og,df], ignore_index=True)
data_og.info()
data_og.head()

data_og.to_csv(r'combined_csvs\full_data.csv', index=False)

C:\Users\vojte\AppData\Local\Temp\ipykernel_23956\4131691959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Country', 'Division']] =df['Div'].str.extract(r'([A-Za-z]+)(\d+)')
C:\Users\vojte\AppData\Local\Temp\ipykernel_23956\4131691959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Country', 'Division']] =df['Div'].str.extract(r'([A-Za-z]+)(\d+)')
C:\Users\vojte\AppData\Local\Temp\ipykernel_23956\4131691959.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         42593 non-null  object
 1   Time         42593 non-null  object
 2   HomeTeam     42593 non-null  object
 3   AwayTeam     42593 non-null  object
 4   FTHG         42593 non-null  int64 
 5   FTAG         42593 non-null  int64 
 6   FTR          42593 non-null  object
 7   Country      42593 non-null  object
 8   Division     42593 non-null  object
 9   Total_goals  42593 non-null  int64 
 10  Target       42593 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 3.6+ MB


# Data encoding and transformation

going to create function  to prepare data by the same key for all the files, full or country by country if chosen to train 11 different models

NOTE: A0 means the model with original data, A1 will be the model with newer data in this file

In [ ]:
'''
Date - idk wtf to do with dat
Time - only keep hour, then standartize ?   range(0,1)
HomeTeam - encode same as away              range(idk)
AwayTeam- Encode same as home               range(idk)
FTHG- get averages, then drop    expected   range(0-6)
FTAG-get averages, then drop     expected   range(0-6)
FTR- get winrate( in 0-1 range) then drop   range(0-1)
Country- dummies, only for full dataset     {0,1}
Division - dummies                          {0,1}
Total_goals- drop in the end i guess
Target - KEEP TARGET                        values:{0,1} Note: 0 is less than 2.5, 1 is more than 2.5
'''
SEED=66


def transf_encode_a0(df):
    #encodery:

    #all this should work generally
    '''
    jestli chcete i transformovat s custom seedem
    if seed is None:
        seed=42
    #misto pro encodery:
    '''
    df['Date']=pd.to_datetime(df['Date'],format='ˇ%d/%m%Y')
    df['Time']= pd.to_datetime(df['Time'],format='%H/%M').dt.hour
    #mean goals calc:
    df=df.sort_values('Date')# sort to make sure averages are for the 5 previous matches
    mean_goals=df.groupby  
    # 'HomeTeam','AwayTeam' same encoding to keep the team numbers same for easiness
    #home
    df['HomeTeam']=#enocde home

    df['avg_goals_in_last5_home']=
    df['avg_goals_conceded_last5_home']=
    df['winrate_last5_home']=
    #away
    df['AwayTeam']=#encode away
    df['avg_goals_in_last5_home']=
    df['avg_goals_conceded_last5_home']=
    df['winrate_last5_home']=
    #this should make it work for the full file - dummies for country, division should be just fine i guess :eshrug:
    if 'Country' in df.columns:
        df=pd.get_dummies(df, columns=['Country','Division'],drop_first=True)
        #encode country/dummies
    else:
        df=pd.get_dummies(df, columns=['Division'],drop_first=True)
        df=df.drop(columns=['Country'])
    
    return df
    


SyntaxError: invalid syntax (1754714230.py, line 17)

transformation for A1

In [ ]:
def transf_encode_a1(df):
    return df